Hybrid: Semantic + Keyword search (dotproduct)

In [1]:
pinecone_api="<your-api-key>"

In [2]:
!pip install pinecone-client pinecone-text pinecone-notebooks

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 7.4 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=c36dea53bf55069bd59e192586abbbba74f932dc111c6e6bd65dac97202a9132
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [3]:
!pip install langchain langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.8/384.8 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [4]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [5]:
from pinecone import Pinecone, ServerlessSpec
index_name="pinecone-hybrid-research-paper-hybrid"
pc=Pinecone(api_key=pinecone_api)
pc.create_index(
    name=index_name,
    dimension=768,
    metric="dotproduct",
    spec=ServerlessSpec(
       cloud="aws",
       region="us-east-1"
    )
)


In [6]:
index=pc.Index(index_name)
index

In [7]:
!pip install --upgrade --quiet  langchain-google-genai

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key="<your-api-key>")
embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x783e5ac7d5a0>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [9]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
retriver=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Step 1: Read the content of the text file
with open('/content/1-s2.0-S0925838824023909-main.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Step 2: Initialize the RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,  # Specify the chunk size (number of characters)
    chunk_overlap=70  # Specify the overlap between chunks
)

# Step 3: Split the text into chunks
chunks = text_splitter.split_text(text)




In [12]:
index=pc.Index(index_name)
index

In [13]:
retriver=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [14]:
retriver.add_texts(chunks)

  0%|          | 0/7 [00:00<?, ?it/s]

In [15]:
#keyword search 1
retriver.invoke("solution treatment has average grain size ranging from")
#score:1

[Document(page_content='sizes and reduced misorientation at grain boundaries. Furthermore,\nsolution treatment alleviates residual stresses, fosters a uniform grain\norientation, and induces changes in crystallographic texture, all of\nwhich impact misorientation patterns and subsequently lower KAM\nvalues.'),
 Document(page_content='3. Results and discussion\n3.1. AR and ST samples\nFigs. 2(a) and 2(b) show the optical microscopy (OM) images of the\nas-received (AR) and solution-treated (ST) samples, respectively.\nFollowing solution treatment at 800◦ C for 1 h, the ST sample reveals an\naverage grain size ranging from 60 to 70 µm. Fig. 2(c) and (d) show the\nND-IPF (normal direction -inverse pole figures) maps of the AR and ST'),
 Document(page_content='samples. The AR sample predominantly features deformed grains oriented along the rolling direction (RD), with a texture characterized by\nthe alignment of (001) poles with the RD. However, microstructural\nhomogenization caused by sol

In [16]:
#keyword search 2
retriver.invoke("authorship contribution")
# score:1

[Document(page_content='CRediT authorship contribution statement\nIsh Kumar Jha: Methodology, Investigation. Rajesh Kisni Khatirkar: Supervision, Investigation. Jaiveer Singh: Writing – review &\nediting, Supervision, Conceptualization. Jaideep Gupta: Writing –\noriginal draft, Investigation.\nDeclaration of Competing Interest\nI confirm that there is no conflict of interest.\nData availability'),
 Document(page_content='of the dependence of mechanical properties on the resultant microstructure [33], [34]. In this work, techniques of optical microscopy,\ntensile testing, EBSD, and fractography have been used to investigate the\nrelationship between microstructural evolution and mechanical properties of Ti-15333 alloy. The current study shall contribute not only to a'),
 Document(page_content='stress on the surrounding matrix, contributing to additional misalignment of neighboring grains resulting in a wider spread of crystallographic orientation. The EBSD observation and analysis condu

In [17]:
#keyword search 3
retriver.invoke("sample aged at 500◦C for 6 h exhibits an UTS of")
# score:1

[Document(page_content='8 h had a UTS of 1209 MPa and elongation of 10 %.\nFurthermore, the sample aged at 500◦ C for 6 h exhibits an UTS of\n1160 MPa, which rises to 1214 MPa after 9 h and 1392 MPa after 15 h\n(see Fig. S2(a)). This corresponds to a 4.6 % and 20 % improvement for\nthe 9 h and 15 h samples, respectively, compared to the 6 h sample. UTS\nfor the sample aged at 550◦ C for 6 h is 1087 MPa, rising to 1255 MPa'),
 Document(page_content='mechanical investigation.\nFor 6 h aged samples, there is a positive correlation between UTS and'),
 Document(page_content='J. Gupta et al.\n\nJournal of Alloys and Compounds 1004 (2024) 175803\n\nFig. 9. Variation of (a) UTS and (b) ductility of 3 h aged samples at various aging temperatures.\n\nFig. 10. Variation of (a) UTS and (b) ductility for 6 h aged samples at various aging temperatures.'),
 Document(page_content='improves both UTS and elongation, achieving a balance of strength\nand ductility. At 500 ◦ C, the alloy reaches 1160 MPa U

In [55]:
#keyword search 4
retriver.invoke("The main aim of this study is ")
# score:1

AttributeError: 'NoneType' object has no attribute 'encode_queries'

In [19]:
#keyword search 5
retriver.invoke("what is the alloy used in this study ?")
# score:1

[Document(page_content='Fig. 1. Thermomechanical processing of Ti-15333 alloy used in this study.'),
 Document(page_content='of the dependence of mechanical properties on the resultant microstructure [33], [34]. In this work, techniques of optical microscopy,\ntensile testing, EBSD, and fractography have been used to investigate the\nrelationship between microstructural evolution and mechanical properties of Ti-15333 alloy. The current study shall contribute not only to a'),
 Document(page_content='[10] O.M. Ivasishin, P.E. Markovsky, Y.V. Matviychuk, S.L. Semiatin, C.H. Ward, S. Fox,\nA comparative study of the mechanical properties of high-strength β-titanium\nalloys, J. Alloy. Compd. 457 (2008) 296–309, https://doi.org/10.1016/j.\njallcom.2007.03.070.'),
 Document(page_content='This study aims to investigate the mechanical behaviors of Ti–15 V–3Cr–3Al–3Sn (Ti-15333), a metastable β\ntitanium alloy. Thermomechanical processing of the alloy sheet in its as-received state included solu

In [20]:
#semantic search 1
retriver.invoke("what is the alloy used in this study ?")
# score:1

[Document(page_content='Fig. 1. Thermomechanical processing of Ti-15333 alloy used in this study.'),
 Document(page_content='of the dependence of mechanical properties on the resultant microstructure [33], [34]. In this work, techniques of optical microscopy,\ntensile testing, EBSD, and fractography have been used to investigate the\nrelationship between microstructural evolution and mechanical properties of Ti-15333 alloy. The current study shall contribute not only to a'),
 Document(page_content='[10] O.M. Ivasishin, P.E. Markovsky, Y.V. Matviychuk, S.L. Semiatin, C.H. Ward, S. Fox,\nA comparative study of the mechanical properties of high-strength β-titanium\nalloys, J. Alloy. Compd. 457 (2008) 296–309, https://doi.org/10.1016/j.\njallcom.2007.03.070.'),
 Document(page_content='This study aims to investigate the mechanical behaviors of Ti–15 V–3Cr–3Al–3Sn (Ti-15333), a metastable β\ntitanium alloy. Thermomechanical processing of the alloy sheet in its as-received state included solu

In [21]:
#semantic search 2
retriver.invoke("what is effect of aging temperature on microstructure of the sample ?")
# score:1

[Document(page_content='4. While aged samples show random texture, the average KAM value\nshows an increase with both the aging temperature and duration,\npeaking in samples aged for 6 h at 500 ◦ C and 550 ◦ C. This is due to\nthe presence of an optimized microstructure at these temperatures,\nfeaturing a substantial amount of well-distributed α phase that can'),
 Document(page_content='on the microstructural alterations associated with aging and precipitation. Fig. 14 (a) and (b) illustrate the change in KAM value at different\naging temperatures for 3 h and 6 h respectively. As the aging time'),
 Document(page_content='improve its strength, the aging process is used to precipitate the secondary α phase evenly throughout the microstructure.\nFigs. 3 and 4 show the microstructures of Ti-15333 alloy as a function of time and temperature. These microstructures were obtained by\nperforming a super transus solution treatment at a temperature of\n800 ◦ C (with βt for Ti-15333 being 760 ◦ C)

In [22]:
#semantic search 3
retriver.invoke("what are the main fractography results of this study ")
# score:1

[Document(page_content='of the dependence of mechanical properties on the resultant microstructure [33], [34]. In this work, techniques of optical microscopy,\ntensile testing, EBSD, and fractography have been used to investigate the\nrelationship between microstructural evolution and mechanical properties of Ti-15333 alloy. The current study shall contribute not only to a'),
 Document(page_content='3.5. Fractography\nThe fracture appearance of Ti 15333 is closely correlated with the\nmicrostructure of the alloy. Fig. 15 shows the different fracture features\nobserved in AR, ST, and aged tensile-tested samples. During the examination of AR and ST alloys, it was observed that the fracture surfaces'),
 Document(page_content='Fig. 16 shows a magnified fractograph of 500 ◦ C 6 h aged fractured\ntensile sample, SEM image reveals a periodic pattern of dimples and\ncleavage, with evenly distributed cleavage facets across the surface,\nwhich may have contributed to its higher ductility. This r

In [23]:
#semantic search 4
retriver.invoke("who all are the authors and what are their contributions to this study ? ")
# score:0.5

[Document(page_content='of the dependence of mechanical properties on the resultant microstructure [33], [34]. In this work, techniques of optical microscopy,\ntensile testing, EBSD, and fractography have been used to investigate the\nrelationship between microstructural evolution and mechanical properties of Ti-15333 alloy. The current study shall contribute not only to a'),
 Document(page_content='CRediT authorship contribution statement\nIsh Kumar Jha: Methodology, Investigation. Rajesh Kisni Khatirkar: Supervision, Investigation. Jaiveer Singh: Writing – review &\nediting, Supervision, Conceptualization. Jaideep Gupta: Writing –\noriginal draft, Investigation.\nDeclaration of Competing Interest\nI confirm that there is no conflict of interest.\nData availability'),
 Document(page_content='I confirm that there is no conflict of interest.\nData availability\nWe have included all the data used in this study'),
 Document(page_content='Fig. 16 shows a magnified fractograph of 500 ◦ C 6 

In [24]:
#semantic search 5
retriver.invoke("what are the step involved in the thermochemical processing ? ")

# score:1

[Document(page_content='through a variety of heat treatments and hot working techniques [8],\n[9]. This typically involves a sequence of steps, including\nthermo-mechanical processing, followed by solution treatment and\naging. Modifying the processing, solutionizing, or aging temperatures'),
 Document(page_content='Fig. 1. Thermomechanical processing of Ti-15333 alloy used in this study.'),
 Document(page_content='of aging on the evolution of microstructure and its effect on the mechanical properties. Initially, the sheet was hot rolled above its BetaTransus temperature (Tβ) i.e. 760◦ C to retain the β phase and to prevent any phase transformation. The entire thermomechanical processing\nof Ti-15333 alloy used in this study is shown in Fig. 1. To prepare for'),
 Document(page_content='and aging, a horizontal tube furnace (Swam Equip, Chennai) was used in\nan environment purged with argon gas to prevent oxidation. The target\ntemperature was reached by heating at a rate of 5 ◦ C per mi

Semantic search

In [29]:
!pip install langchain langchain-community

In [32]:
pip install --upgrade langchain-pinecone


In [34]:
from pinecone import Pinecone, ServerlessSpec
index_name="pinecone-hybrid-research-paper-semantic"
pc=Pinecone(api_key=pinecone_api)
pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(
       cloud="aws",
       region="us-east-1"
    )
)
index=pc.Index(index_name)



In [37]:
!pip install langchain-pinecone

In [46]:
from langchain.vectorstores import Pinecone
import os

In [47]:
os.environ['PINECONE_API_KEY'] = '<your-api-key>'

In [48]:
vectorstore = Pinecone.from_texts(
    texts=chunks,
    embedding=embeddings,
    index_name=index_name
)

In [50]:
#keyword search 1
vectorstore.similarity_search("solution treatment has average grain size ranging from")

#score:1


[Document(page_content='3. Results and discussion\n3.1. AR and ST samples\nFigs. 2(a) and 2(b) show the optical microscopy (OM) images of the\nas-received (AR) and solution-treated (ST) samples, respectively.\nFollowing solution treatment at 800◦ C for 1 h, the ST sample reveals an\naverage grain size ranging from 60 to 70 µm. Fig. 2(c) and (d) show the\nND-IPF (normal direction -inverse pole figures) maps of the AR and ST'),
 Document(page_content='solution treatment and aging, small samples were obtained using a\nprecision diamond-cutting machine (Metco, Chennai). These samples\nmeasured 5 mm in both the RD and TD directions.\nFollowing a 1 h solution treatment at 800 ◦ C, the samples underwent\naging treatments at temperatures ranging from 450 ◦ C to 700 ◦ C for\ndifferent durations (1 h, 3 h, and 6 h). To carry out solution treatment'),
 Document(page_content='samples. The AR sample predominantly features deformed grains oriented along the rolling direction (RD), with a texture cha

In [51]:
#keyword search 2
vectorstore.similarity_search("authorship contribution")
#score:1

[Document(page_content='CRediT authorship contribution statement\nIsh Kumar Jha: Methodology, Investigation. Rajesh Kisni Khatirkar: Supervision, Investigation. Jaiveer Singh: Writing – review &\nediting, Supervision, Conceptualization. Jaideep Gupta: Writing –\noriginal draft, Investigation.\nDeclaration of Competing Interest\nI confirm that there is no conflict of interest.\nData availability'),
 Document(page_content='I confirm that there is no conflict of interest.\nData availability\nWe have included all the data used in this study'),
 Document(page_content='https://doi.org/10.1016/j.jallcom.2024.175803\nReceived 2 May 2024; Received in revised form 21 July 2024; Accepted 31 July 2024\nAvailable online 31 July 2024\n0925-8388/© 2024 Elsevier B.V. All rights are reserved, including those for text and data mining, AI training, and similar technologies.'),
 Document(page_content='J. Gupta et al.\n\nJournal of Alloys and Compounds 1004 (2024) 175803')]

In [52]:
#keyword search 3
vectorstore.similarity_search("sample aged at 500◦C for 6 h exhibits an UTS of")
#score:1


[Document(page_content='8 h had a UTS of 1209 MPa and elongation of 10 %.\nFurthermore, the sample aged at 500◦ C for 6 h exhibits an UTS of\n1160 MPa, which rises to 1214 MPa after 9 h and 1392 MPa after 15 h\n(see Fig. S2(a)). This corresponds to a 4.6 % and 20 % improvement for\nthe 9 h and 15 h samples, respectively, compared to the 6 h sample. UTS\nfor the sample aged at 550◦ C for 6 h is 1087 MPa, rising to 1255 MPa'),
 Document(page_content='mechanical investigation.\nFor 6 h aged samples, there is a positive correlation between UTS and'),
 Document(page_content='peak aging and there are apparent precipitation-free zones in the OM\nimages. Therefore, at the peak aging temperatures of 500 ◦ C and 550 ◦ C,\naging time has been further increased to 9 h and 15 h. Supplementary\nFig. S1(a-b) shows OM images of samples aged at 500 ◦ C for 9 h and\n15 h, respectively. There is a significant reduction in the size of'),
 Document(page_content='J. Gupta et al.\n\nJournal of Alloys and Com

In [54]:
#keyword search 4
vectorstore.similarity_search("The main aim of this study is ")
#score:1

[Document(page_content='CRediT authorship contribution statement\nIsh Kumar Jha: Methodology, Investigation. Rajesh Kisni Khatirkar: Supervision, Investigation. Jaiveer Singh: Writing – review &\nediting, Supervision, Conceptualization. Jaideep Gupta: Writing –\noriginal draft, Investigation.\nDeclaration of Competing Interest\nI confirm that there is no conflict of interest.\nData availability'),
 Document(page_content='of the dependence of mechanical properties on the resultant microstructure [33], [34]. In this work, techniques of optical microscopy,\ntensile testing, EBSD, and fractography have been used to investigate the\nrelationship between microstructural evolution and mechanical properties of Ti-15333 alloy. The current study shall contribute not only to a'),
 Document(page_content='better understanding of the relationship between the microstructure and\nthe mechanical properties of Ti-15333 but also to its precise control and\nprediction.\n2. Materials and methods\nIn the pr

In [62]:
#keyword search 5
vectorstore.similarity_search("what is this text about?")
#score:1

[Document(page_content='of the dependence of mechanical properties on the resultant microstructure [33], [34]. In this work, techniques of optical microscopy,\ntensile testing, EBSD, and fractography have been used to investigate the\nrelationship between microstructural evolution and mechanical properties of Ti-15333 alloy. The current study shall contribute not only to a'),
 Document(page_content='Journal of Alloys and Compounds 1004 (2024) 175803\n\nContents lists available at ScienceDirect\n\nJournal of Alloys and Compounds\njournal homepage: www.elsevier.com/locate/jalcom\n\nEffect of aging on tensile and fracture behavior of a metastable\nTi-15 V–3Cr–3Al–3Sn β-titanium alloy\nJaideep Gupta a , Ish Kumar Jha a , Rajesh K. Khatirkar b , Jaiveer Singh a, *\na\nb'),
 Document(page_content='Sponsored by the Titanium Committee of TMS, Held at the 1993 Annual TMS\nMeeting, Warrendale, PA (United States), Minerals, Metals and Materials Society,\nDenver,Colorado, 1993.\n[33] C. Li, J. Che

In [57]:
#semantic search 1
vectorstore.similarity_search("what is the alloy used in this study ?")
#score:1

[Document(page_content='Fig. 1. Thermomechanical processing of Ti-15333 alloy used in this study.'),
 Document(page_content='This study aims to investigate the mechanical behaviors of Ti–15 V–3Cr–3Al–3Sn (Ti-15333), a metastable β\ntitanium alloy. Thermomechanical processing of the alloy sheet in its as-received state included solution treatment at 800◦ C, followed by varying aging temperatures from 450◦ C to 700◦ C. As the aging temperature rises, α'),
 Document(page_content='of the dependence of mechanical properties on the resultant microstructure [33], [34]. In this work, techniques of optical microscopy,\ntensile testing, EBSD, and fractography have been used to investigate the\nrelationship between microstructural evolution and mechanical properties of Ti-15333 alloy. The current study shall contribute not only to a'),
 Document(page_content='better understanding of the relationship between the microstructure and\nthe mechanical properties of Ti-15333 but also to its precise cont

In [58]:
#semantic search 2
vectorstore.similarity_search("what is effect of aging temperature on microstructure of the sample ?")
#score:1

[Document(page_content='tensile specimens using EBSD revealed a random texture. KAM value reflecting dislocation density rose with both\naging temperature and duration, with its highest average reported for 6 h aging condition at 500◦ C and 550◦ C,\naligning with peak level of α precipitation. The size, shape, and size distributions of α precipitates also had a'),
 Document(page_content='limited ductility in such instances. As the aging temperature rose to\n600◦ C and above, over-aged specimens predominantly displayed dimple\nfracture rather than quasi-cleavage. The emergence of large dimples was\nattributed to the presence of coarse α platelets in the over-aged specimens. These findings also align with the observations made during the\nmechanical investigation.'),
 Document(page_content='4. While aged samples show random texture, the average KAM value\nshows an increase with both the aging temperature and duration,\npeaking in samples aged for 6 h at 500 ◦ C and 550 ◦ C. This is due t

In [59]:
#semantic search 3
vectorstore.similarity_search("what are the main fractography results of this study ")
#score:1


[Document(page_content='Fig. 15. Different fracture features observed in aged tensile samples, (a) and (b) AR, (c) and (d) ST (e) 500◦ C 3 h and (f) 500◦ C 6 h.\n11\n\n\x0cJ. Gupta et al.\n\nJournal of Alloys and Compounds 1004 (2024) 175803\n\nFig. 16. Magnified fractograph of 500◦ C/6 h aged fractured tensile sample shows alternate patterns of dimples and cleavage facets.'),
 Document(page_content='ductility with an increase in temperature from 450 ◦ C and 500 ◦ C, as has\nbeen previously reported in this paper. This unusual behavior of Ti15333 alloy at 500 ◦ C 6 h can be explained by comparing the fractograph of samples aged at this condition to those aged at other conditions.\nFig. 16 shows a magnified fractograph of 500 ◦ C 6 h aged fractured'),
 Document(page_content='Fig. 16 shows a magnified fractograph of 500 ◦ C 6 h aged fractured\ntensile sample, SEM image reveals a periodic pattern of dimples and\ncleavage, with evenly distributed cleavage facets across the surface,\nwhich 

In [60]:
#semantic search 4
vectorstore.similarity_search("who all are the authors and what are their contributions to this study ?")
#score:1

[Document(page_content='CRediT authorship contribution statement\nIsh Kumar Jha: Methodology, Investigation. Rajesh Kisni Khatirkar: Supervision, Investigation. Jaiveer Singh: Writing – review &\nediting, Supervision, Conceptualization. Jaideep Gupta: Writing –\noriginal draft, Investigation.\nDeclaration of Competing Interest\nI confirm that there is no conflict of interest.\nData availability'),
 Document(page_content='I confirm that there is no conflict of interest.\nData availability\nWe have included all the data used in this study'),
 Document(page_content='Department of Metallurgical and Materials Engineering, Indian Institute of Technology Jodhpur, Karwar, Jodhpur, Rajasthan 342037, India\nDepartment of Metallurgical and Materials Engineering, Visvesvaraya National Institute of Technology, South Ambazari Road, Nagpur, Maharashtra 440010, India\n\nA R T I C L E I N F O\n\nA B S T R A C T'),
 Document(page_content='of the dependence of mechanical properties on the resultant micro

In [61]:
#semantic search 5
vectorstore.similarity_search("what are the step involved in the thermochemical processing ?")
#score:1

[Document(page_content='through a variety of heat treatments and hot working techniques [8],\n[9]. This typically involves a sequence of steps, including\nthermo-mechanical processing, followed by solution treatment and\naging. Modifying the processing, solutionizing, or aging temperatures'),
 Document(page_content='of aging on the evolution of microstructure and its effect on the mechanical properties. Initially, the sheet was hot rolled above its BetaTransus temperature (Tβ) i.e. 760◦ C to retain the β phase and to prevent any phase transformation. The entire thermomechanical processing\nof Ti-15333 alloy used in this study is shown in Fig. 1. To prepare for'),
 Document(page_content='and aging, a horizontal tube furnace (Swam Equip, Chennai) was used in\nan environment purged with argon gas to prevent oxidation. The target\ntemperature was reached by heating at a rate of 5 ◦ C per minute, and the\ntemperature was held steady within ± 1 ◦ C during the entire process.\nOptical microsc

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from pinecone import Pinecone, ServerlessSpec

# Initialize embeddings
embeddings = HuggingFaceEmbeddings()

# Initialize Pinecone index
index_name = "pinecone-research-paper-semantic"  # Choose a suitable name
pc = Pinecone(api_key=pinecone_api)
#If index already exists, delete it and create a new one
if index_name in pc.list_indexes():
    pc.delete_index(index_name)
pc.create_index(
    name=index_name,
    dimension=768,  # Adjust to match your embedding dimension
    metric="cosine",
    spec=ServerlessSpec(
       cloud="aws",
       region="us-east-1"
    )
)
index = pc.Index(index_name)

# Initialize Pinecone vectorstore
text_field= "text"
vectorstore = Pinecone.from_texts(
    texts=chunks,
    embedding=embeddings,
    index_name=index_name
)

# Create a retriever from the vectorstore
retriever = vectorstore.as_retriever()

# Now you can use the retriever for semantic search
retriever.get_relevant_documents("your query here")